In [1]:
import os
import math
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import itertools as it
from scipy.ndimage import correlate, gaussian_filter
import pydicom as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
def imgtrim(img, c):
    img[img < -c] = -c
    img[img > c] = c
    return img

def imgpow(img, a, b, c, vmax = 0):
    img[img < a] = a
    img[img > b] = b
    img -= np.min(img)
    if vmax == 0:
        img = img / (b-a)
    else:
        img = img / np.max([b-a, vmax])
    img = np.power(img, c)
    return img

In [3]:
class DICOM:
    """Esta classe é usada para armazenar a imagem importada usando a biblioteca Pydicom.
    Tanto uma única imagem Dicom como uma pasta correspondente a uma imagem tridimensional podem ser carregados.
    No caso tridimensional, as fatias são ordenadas e disponibilizadas como um tensor de ordem 3 em self.pixel_array"""
    # Os códigos de tag abaixo são definidos pelo padrão DICOM
    _img_pos = pd.tag.Tag(0x00200032)
    _img_ori = pd.tag.Tag(0x00200037)
    _pct_pos = pd.tag.Tag(0x00185100)
    _slice_thickness = pd.tag.Tag(0x00180050)
    _pixel_spacing = pd.tag.Tag(0x00280030)
    
    def __init__(self, path, d3 = False, file_ext = ".dcm"):
        self.d3 = d3
        self.file_ext = file_ext
        self.path = path
        
        if not d3:
            self.dcm_array = [pd.dcmread(path)]
            self.pixel_array = np.array([self.dcm_array[0].pixel_array])
        else:
            dcm_array = []
            for filename in os.listdir(path):
                if filename.endswith(file_ext):
                    dcm_array.append(pd.dcmread((os.path.join(path, filename))))
            dcm_array.sort(key = lambda x: x[DICOM._img_pos].value[2])
            self.dcm_array = dcm_array
            
            pixel_array = []
            for dcm in dcm_array:
                pixel_array.append(dcm.pixel_array)
            self.pixel_array = np.array(pixel_array)
            #self.padded_array = np.pad(pixel_array, ((0,2),(0,0),(0,0)), 'constant', constant_values = np.min(pixel_array))
            #self.pixel_array = self.padded_array[1:-1,:,:]
        self.res = np.array(list(self.dcm_array[0][DICOM._pixel_spacing].value)
                            + [self.dcm_array[0][DICOM._slice_thickness].value], dtype='float')
        self.shape = self.pixel_array.shape
        self.trim()
    
    def trim(self, plt_lower = -256, plt_upper = 256, plt_pow = 1):
        self.plt_lower, self.plt_upper, self.plt_pow = plt_lower, plt_upper, plt_pow
        self.trimmed_array = imgpow(np.array(self.pixel_array), plt_lower, plt_upper, plt_pow, 0)
    """
    def calcslice(self, b = 0, ax = 0, ay = 0):
        nz, nx, ny = self.shape
        resx, resy, resz = self.res
        ax = resx/resz * ax
        ay = resy/resz * ay
        b = b/resz
        minx = math.ceil((-b/ax)*(1 + ax ** 2) ** .5) if ax != 0 else 0
        miny = math.ceil((-b/ay)*(1 + ay ** 2) ** .5) if ay != 0 else 0 
        maxx = -1 + math.ceil(-b + nx * (1 + ax ** 2) ** .5) if ax < nz/nx else -1 + math.ceil(nz * (1 + ax ** -2) ** .5)
        maxy = -1 + math.ceil(-b + ny * (1 + ay ** 2) ** .5) if ay < nz/ny else -1 + math.ceil(nz * (1 + ay ** -2) ** .5)
        print(minx, maxx, miny, maxy)
        x = np.repeat(range(2*minx, 2*maxx), 2*(maxy-miny)).reshape(2*(maxx-minx), 2*(maxy-miny))
        y = np.transpose(np.repeat(range(2*miny, 2*maxy), 2*(maxx-minx)).reshape(2*(maxy-miny), 2*(maxx-minx)))
        x = (x / (1 + ax ** 2) ** .5)/2
        y = (y / (1 + ay ** 2) ** .5)/2
        z = b + ax*x + ay*y
        z[z > nz-1] = nz
        z[z < 0] = nz
        
        result = np.zeros(x.shape)
        x0, y0, z0 = np.floor([x, y, z]).astype('int64')
        x1, y1, z1 = [x0+1, y0+1, z0+1]
        dsum = np.zeros(x.shape)
        for r in it.product([0,1],[0,1],[0,1]):
            d = ((resx*(r[0] + x0 - x))**2 + (resy*(r[1] + y0 - y))**2 + (resz*(r[2] + z0 - z))**2) ** .5 + 0.0001
            result += self.padded_array[r[2] + z0, r[0] + x0, r[1] + y0] / d
            dsum += 1 / d
        result = result / dsum
        
        nresx = resx*(1 + ax**2)**.5 if ax <= resz/resx else resz*(1 + ax**-2)**.5
        nresy = resy*(1 + ay**2)**.5 if ay <= resz/resy else resz*(1 + ay**-2)**.5
        print(result.shape)
        print(z)
        plt.figure(num=None, figsize=(8, 8), dpi=80, facecolor='w', edgecolor='k')
        plt.set_cmap('Greys_r')
        plt.imshow(result, aspect = nresx/nresy)
        plt.plot()
        
        return result
    """
    def plot(self, img):
        plt.figure(num=None, figsize=(8, 8), dpi=80, facecolor='w', edgecolor='k')
        plt.set_cmap('Greys_r')
        plt.imshow(img)
        plt.plot();
    
    def plotslice(self, z = 0, setparams = False, a = -256, b = 256, c = 1):
        imgO = np.array(self.pixel_array[z])
        if not setparams:
            imgO = imgpow(imgO, self.plt_lower, self.plt_upper, self.plt_pow, np.max(self.pixel_array))
        else:
            imgO = imgpow(imgO, a, b, c, np.max(self.pixel_array))
        self.plot(imgO)
    
    def startinteractiveplot(self, tensor = None, trimmed = True):
        if tensor is None:
            if trimmed:
                self.plt_array = imgpow(np.array(self.pixel_array),
                                        self.plt_lower, self.plt_upper, self.plt_pow, 0)
            else: 
                self.plt_array = np.array(self.pixel_array)
                self.plt_array -= np.min(self.plt_array)
                self.plt_array = self.plt_array/np.max(self.plt_array)
        else:
            self.plt_array = np.array(tensor)
            self.plt_array -= np.min(self.plt_array)
            self.plt_array = self.plt_array/np.max(self.plt_array)
            
        self.fig = plt.figure(facecolor='w', edgecolor='k')
        plt.set_cmap('Greys_r')
        self.ax = self.fig.add_subplot(1,1,1)
        if self.plt_array.ndim == 3:
            self.im = self.ax.imshow(self.plt_array[0], vmin=0.0, vmax=1.0)
        elif self.plt_array.ndim == 4:
            self.im = self.ax.imshow(self.plt_array[(0,0)], vmin=0.0, vmax=1.0)
    
    def interactiveplot(self, z = 0, pixel_data = None):
        if pixel_data is None:
            self.im.set_data(self.plt_array[z])
        else:
            self.im.set_data(pixel_data)
        self.im.axes.figure.canvas.draw()
    
    def interact(self, tensor = None, dynamic_gamma = False, trimmed = True):
        if not dynamic_gamma:
            self.startinteractiveplot(tensor, trimmed = trimmed)
            if (self.plt_array.ndim == 3):
                interact(lambda z: self.interactiveplot(z),
                         z = widgets.IntSlider(min=0,max=-1+self.plt_array.shape[0],step=1,value=0))
            elif (self.plt_array.ndim == 4):
                interact(lambda t,z: self.interactiveplot((t,z)),
                         t = widgets.IntSlider(min=0,max=-1+self.plt_array.shape[0],step=1,value=0),
                         z = widgets.IntSlider(min=0,max=-1+self.plt_array.shape[1],step=1,value=0))
        else:
            self.startinteractiveplot()
            interact(lambda z, a, b, c: self.interactiveplot(pixel_data = imgpow(np.array(self.pixel_array[z]), a, b, c, 0)),
                     z=widgets.IntSlider(min=0,max=len(self.dcm_array)-1,step=1,value=0),
                     a=widgets.IntSlider(min=-2048,max=8192,step=1,value=-256),
                     b=widgets.IntSlider(min=-2048,max=8192,step=1,value=256),
                     c=widgets.FloatSlider(min=0.1,max=2,step=0.1,value=1)
                    );

In [4]:
path = "C:/Users/Demetrius/Desktop/APOLLO/AP-6H6G/09-09-2015-CT THORAX WCONT-94638/3-Body 2.0 WITH CONT CE-62197"
dicom = DICOM(path, d3 = True)

#list(dcm_array[0][DICOM._pixel_spacing].value) + [dcm_array[0][DICOM._slice_thickness].value]

In [14]:
%matplotlib
dicom.interact(dynamic_gamma = True)
#dicom.plotslice()

Using matplotlib backend: Qt5Agg


interactive(children=(IntSlider(value=0, description='z', max=157), IntSlider(value=-256, description='a', max=8192, min=-2048), IntSlider(value=256, description='b', max=8192, min=-2048), FloatSlider(value=1.0, description='c', max=2.0, min=0.1), Output()), _dom_classes=('widget-interact',))

In [5]:
class Dirac:
    def __init__(self, epsilon):
        self.epsilon = epsilon
    
    def __call__(self, M):
        pass

class Heaviside:
    def __init__(self, epsilon):
        self.epsilon = epsilon
    
    def __call__(self, M):
        pass    
    
class D_atan(Dirac):
    def __init__(self, epsilon):
        super().__init__(epsilon)

    def __call__(self, M):
        M = np.array(M)
        return self.epsilon * ((self.epsilon**2 + M**2) ** -1)

class H_atan(Heaviside):
    def __init__(self, epsilon):
        super().__init__(epsilon)
        
    def __call__(self, M):
        M = np.array(M)
        return .5 + np.arctan(M/self.epsilon) / np.pi
    
class H_0(Heaviside):
    def __init__(self, epsilon):
        super().__init__(epsilon)
        
    def __call__(self, M):
        R = np.zeros(M.shape)
        R[M > 0] = 1
        return R

In [6]:
dirac = D_atan(.5)
heaviside = H_atan(.5)
h0 = H_0(0)

In [7]:
class Segmentation:
    def __init__(self, dt, dirac, heaviside, dicom, 
                 z = -1, k=[1, .1, 1, 1], pregaussian = 0, 
                 lowmem = False, phi0 = 'sine'):
        self.dt = dt
        self.dirac = dirac
        self.heaviside = heaviside
        self.dicom = dicom
        self.lowmem = lowmem
        if z == -1:
            image = np.array(self.dicom.trimmed_array)
            image = image.astype('float64', casting='safe', copy = True)
        else:
            image = np.array(self.dicom.trimmed_array[z])
            image = image.astype('float64', casting='safe', copy = True)
        image = gaussian_filter(image, pregaussian)
        self.image = image
        self.phi0(t = phi0)
        self.k = k
        self.nstep = 0
    
    def phi0(self, t = 'median'):
        self.phi = -1*np.ones(self.image.shape)
        self.ndim = self.image.ndim
        if t == 'median':
            self.phi[self.image > np.median(self.image)] = 1
            self.phi = gaussian_filter(self.phi, 1)
        elif t == 'checkers':
            if self.ndim == 3:
                self.phi[1::2, ::2, ::2] = 1
                self.phi[::2, 1::2, ::2] = 1
                self.phi[::2, ::2, 1::2] = 1
                self.phi[1::2, 1::2, 1::2] = 1
            else:
                self.phi[1::2, ::2] = 1
                self.phi[::2, 1::2] = 1
        elif t == 'sine':
            r = 200
            if self.ndim == 3:
                z, x, y = self.phi.shape
                zp, xp, yp = np.array(self.phi.shape)/r
                for i in range(z):
                    for j in range(x):
                        for k in range(y):
                            self.phi[i, j, k] = math.sin(i/zp) + math.sin(j/xp) + math.sin(k/yp)
            elif self.ndim == 2:
                x, y = self.phi.shape
                xp, yp = np.array(self.phi.shape)/r
                for i in range(x):
                    for j in range(y):
                            self.phi[i, j] = math.sin(i/xp) + math.sin(j/yp)
        self.phi = [self.phi]
    
    """Derivada direcional do tensor.
    Parâmetro d determina direção (x,y,z...) = (0,1,2,...).
    Parâmetro w determina tipo de estimativa (esquerda, centro, direita) = (-1,0,1)."""
    def D(self, d, w, tensor):
        kernel = np.array([0]*(w == 1) + [-1] + [0]*(w==0) + [1] + [0]*(w==-1))
        kernel = np.reshape(kernel, [1]*(self.ndim-d-1) + [3] + [1]*d)
        kernel = kernel / self.dicom.res[d]
        if w == 0:
            kernel = kernel * .5
        return correlate(tensor, kernel, mode='nearest')
    
    def innersum(self):
        return 1.0 * np.sum(self.image * self.heaviside(self.phi[-1]))
    
    def innerarea(self):
        return 1.0 * np.sum(self.heaviside(self.phi[-1]))
    
    def outersum(self):
        return 1.0 * np.sum(self.image * (-1*self.heaviside(self.phi[-1]) + 1))
    
    def outerarea(self):
        return 1.0 * np.sum(-1*self.heaviside(self.phi[-1]) + 1)
    
    def innermean(self):
        return self.innersum()/self.innerarea()

    def outermean(self):
        return self.outersum()/self.outerarea()
    
    def I(self):
        return (self.image - self.innermean())**2
    
    def O(self):
        return (self.image - self.outermean())**2
    
    def divunitgrad(self, tau = .000001):
        Dxic = []
        Dxid = []
        for i in range(self.ndim):
            Dxic.append(self.D(i,0,self.phi[-1]))
            Dxid.append(self.D(i,1,self.phi[-1]))
        r = np.zeros(self.image.shape)
        for i in range(self.ndim):
            gradnorm = tau
            for j in range(self.ndim):
                if i == j:
                    gradnorm += Dxid[j]**2
                else:
                    gradnorm += Dxic[j]**2
            gradnorm = gradnorm ** .5
            r += self.D(i, -1, Dxid[i]/gradnorm)
        return r
    
    def step(self):
        self.nstep += 1
        newphi = self.phi[-1] + self.dt * self.dirac(self.phi[-1]) * (
                    - self.k[1] 
                    - self.k[2] * self.I() 
                    + self.k[3] * self.O()
                    + self.k[0] * self.divunitgrad()
                )
        if (self.lowmem) & (len(self.phi) > 1):
            del self.phi[-1]
        self.phi.append(newphi)
    
    def steps(self, n = 1, timelimit = 0):
        i = 0
        t0 = time.time()
        while (i < n) & ((timelimit == 0) | (time.time() - t0 < timelimit)):
            self.step()
            i += 1
        print("Total iterations done: {}".format(i))
        print("Current iteration: {}".format(self.nstep))
        print("Total time: {:2.2}".format(time.time() - t0))

In [8]:
%matplotlib
#dicom.trim(np.min(dicom.pixel_array), np.max(dicom.pixel_array[98]), 1)
dicom.trim(-256, 256, 1)
#dicom.interact(dicom.trimmed_array)

Using matplotlib backend: Qt5Agg


In [22]:
seg = Segmentation(.1, dirac, heaviside, dicom, z=98, k=[.01, 0, 10, 10],
                  pregaussian = 0, lowmem = False, phi0='sine')

In [23]:
seg.steps(50)

Total iterations done: 50
Current iteration: 50
Total time: 8.2


In [24]:
%matplotlib
plt_arr = h0(np.array(seg.phi))
dicom.interact(plt_arr)
#dicom.interact(seg.image)

Using matplotlib backend: Qt5Agg


interactive(children=(IntSlider(value=0, description='z', max=50), Output()), _dom_classes=('widget-interact',))

In [187]:
%matplotlib
dicom.interact(dicom.trimmed_array)

Using matplotlib backend: Qt5Agg


interactive(children=(IntSlider(value=0, description='z', max=157), Output()), _dom_classes=('widget-interact'…